In [18]:
import copy
from tqdm.auto import tqdm
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


from superprimitive_fusion.scanner import (
    virtual_mesh_scan,
    mesh_depth_image,
    generate_rgbd_noise,
    clean_mesh_and_remap_weights,
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
    distinct_colours,
)
from superprimitive_fusion.mesh_fusion import (
    fuse_meshes,
)
from superprimitive_fusion.mesh_fusion_utils import (
    get_mesh_components,
)

In [8]:
names = (
    ('mustard-bottle', 'mustard-bottle.obj'),
    ('table', 'table.obj'),
    ('power-drill', 'power-drill.obj'),
    ('bleach', 'bleach.obj'),
    ('pitcher', 'pitcher.obj'),
    ('mug', 'mug.obj'),
    ('extra-large-clamp', 'extra-large-clamp-leaning.obj'),
)

gt_meshes = dict()
for foldername,filename in names:
    print(f'Getting the {foldername}')
    
    gt_mesh = o3d.io.read_triangle_mesh(f"../data/posed-meshes/{foldername}/{filename}", enable_post_processing=True)

    bake_uv_to_vertex_colours(gt_mesh)
    gt_mesh.compute_vertex_normals()

    gt_meshes[foldername] = gt_mesh

gt_mesh_list = list(gt_meshes.values())

Getting the mustard-bottle
Getting the table
Getting the power-drill
Getting the bleach
Getting the pitcher
Getting the mug
Getting the extra-large-clamp


In [9]:
centres = []
for gt_meshname, gt_mesh in gt_meshes.items():
    if gt_meshname == 'table':
        continue
    centres.append(gt_mesh.get_center())

centres = np.vstack(centres)

obj_centre = centres.mean(axis=0)

In [10]:
cam_centre_offset=np.array([0, 0, 0.2])
look_at = obj_centre
width_px: int = 360
height_px: int = 240
fov: float = 70.0
k: float = 3.5
max_normal_angle_deg = None

In [15]:
scans = []
N = 10
for theta in tqdm(np.linspace(0,360/N * (N-1), N), desc='Scanning'):
# for theta in np.linspace(30,180, N):
# for theta in [70, 110]:
    object_meshes, object_weights = virtual_mesh_scan(
        meshlist=gt_mesh_list,
        cam_centre=cam_centre_offset + polar2cartesian(0.8, 30, theta),
        look_at=look_at,
        k=10,
        max_normal_angle_deg=None,
        linear_depth_sigma=0,
        quadrt_depth_sigma=0,
        sigma_floor=1e-4,
        bias_k1=0.025,
    )
    scans.append([(m,w) for m,w in zip(object_meshes, object_weights)])

meshlists = [[scan_obj[0] for scan_obj in scan] for scan in scans]
meshes = np.array(meshlists).flatten().tolist()

for mesh in meshes:
    mesh.compute_vertex_normals()
o3d.visualization.draw_geometries(meshes)

Scanning: 100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [16]:
fused_scan = scans[0]
N = len(scans[0])
assert len(set([len(scan) for scan in scans])) == 1

for t in tqdm(range(1,len(scans)), desc='Scan'):
    fused_scan_meshes = []
    fused_scan_weights = []
    for obj_id in tqdm(range(N), desc='Object'):
        mesh1, weights1 = fused_scan[obj_id]
        mesh2, weights2 = scans[t][obj_id]

        if len(np.asarray(mesh1.vertices)) == 0:
            # Either one or both meshes are empty
            print(f'Scan {t-1} obj {obj_id} is empty')
            fused_obj_mesh    = mesh2
            fused_obj_weights = weights2
        elif len(np.asarray(mesh2.vertices)) == 0:
            print(f'Scan {t} obj {obj_id} is empty')
            # mesh2 is bad but mesh1 is fine
            fused_obj_mesh    = mesh1
            fused_obj_weights = weights1
        else:
            # both meshes fine
            # print(f'Fusing obj {obj_id} scans {t-1,t}')
            fused_obj_mesh, fused_obj_weights = fuse_meshes(
                mesh1           = mesh1,
                weights1        = weights1,
                mesh2           = mesh2,
                weights2        = weights2,
                h_alpha         = 2.5,
                r_alpha         = 2.0,
                nrm_shift_iters = 2,
                nrm_smth_iters  = 1,
                shift_all       = False,
                fill_holes      = False,
            )
        fused_scan_meshes.append(fused_obj_mesh)
        fused_scan_weights.append(fused_obj_weights)
        
    fused_scan = [(m,w) for m,w in zip(fused_scan_meshes, fused_scan_weights)]

Scan:   0%|          | 0/9 [00:00<?, ?it/s]

Scan 0 obj 0 is empty


Scan:  22%|██▏       | 2/9 [00:17<01:01,  8.84s/it]

point 21709 has no neighbours
point 21709 has no neighbours


Scan:  33%|███▎      | 3/9 [00:28<00:58,  9.82s/it]

...

Scan 8 obj 0 is empty


point 24649 has no neighbours
point 24649 has no neighbours


Scan:  89%|████████▉ | 8/9 [01:30<00:12, 12.58s/it]

point 40874 has no neighbours
point 40874 has no neighbours


Scan: 100%|██████████| 9/9 [01:46<00:00, 11.85s/it]


In [19]:
fused_scan = scans[0]
N = len(scans[0])
assert all(len(s) == N for s in scans), "All scans must have the same length"

for t, scan in enumerate(tqdm(scans[1:], desc="Scan", unit="scan"), start=1):
    fused_scan_meshes = []
    fused_scan_weights = []
    for obj_id in tqdm(range(N), desc="Object", unit="obj", position=1, leave=False):
        mesh1, weights1 = fused_scan[obj_id]
        mesh2, weights2 = scan[obj_id]

        if len(np.asarray(mesh1.vertices)) == 0:
            tqdm.write(f"Scan {t-1} obj {obj_id} is empty")
            fused_obj_mesh, fused_obj_weights = mesh2, weights2
        elif len(np.asarray(mesh2.vertices)) == 0:
            tqdm.write(f"Scan {t} obj {obj_id} is empty")
            fused_obj_mesh, fused_obj_weights = mesh1, weights1
        else:
            fused_obj_mesh, fused_obj_weights = fuse_meshes(
                mesh1=mesh1, weights1=weights1,
                mesh2=mesh2, weights2=weights2,
                h_alpha=3.5, r_alpha=2.0,
                nrm_shift_iters=2, nrm_smth_iters=1,
                shift_all=False, fill_holes=False,
            )
        fused_scan_meshes.append(fused_obj_mesh)
        fused_scan_weights.append(fused_obj_weights)

    fused_scan = list(zip(fused_scan_meshes, fused_scan_weights))


Scan:   0%|          | 0/9 [00:00<?, ?scan/s]

Object:   0%|          | 0/7 [00:00<?, ?obj/s]

Scan 0 obj 0 is empty


Object:   0%|          | 0/7 [00:00<?, ?obj/s]

Object:   0%|          | 0/7 [00:00<?, ?obj/s]

...

Scan 7 obj 3 is empty
point 16163 has no neighbours
point 16163 has no neighbours
Scan 7 obj 5 is empty


Object:   0%|          | 0/7 [00:00<?, ?obj/s]

Scan 8 obj 0 is empty
point 24694 has no neighbours
point 24694 has no neighbours


Object:   0%|          | 0/7 [00:00<?, ?obj/s]

point 40368 has no neighbours
point 40368 has no neighbours


In [20]:
fused_mesh_result = [obj[0] for obj in fused_scan]
o3d.visualization.draw_geometries(fused_mesh_result)